In [1]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda import gpuarray
from pycuda.compiler import SourceModule

Построим граф всех победных комбинаций выбранного игрока настольной игры Okiya.  
Комбинации поражения в граф включать не будем, для экономии памяти.  
У каждой комбинации на своем ходе, будет вес, соответствующий количеству победных комбинаций в направлении финального хода.  
Так мы сможем сортировать комбинации хода, для выбора наиболее богатой потенциальными победами комбинации.

В игре два игрока. У каждого игрока свой граф побед.  
Размер поля 4 на 4 клетки.  
Возможно 16 ходов, по 8 на каждого игрока.  
Первым ходит игрок 0, Вторым игрок 1.

In [2]:
[1 if i%2 else 0 for i in range(16)]

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

Последним ходит игрок 1. Определим все возможные, победные комбинации для текущего игрока на этапе последнего хода.  
Для поля 4 на 4 возможно 2**(4 * 4) что соответствует 65536 комбинаций.  
Используем 65536 потоков GPU. Каждый поток - уникальная комбинация поля.  
256 блоков в 256 сетках.
Каждая комбинация предствалена в виде int числа от 0 до 65536. Для представления в виде фишек на поле, переведем его в bin, разделим последовательность чисел на 4 части, разместив сверху вниз, например:  
0 1 0 1  
0 1 0 1  
0 1 0 1  
0 1 0 1  

In [3]:
ker = SourceModule("""
__device__ bool win_combination(int i)
{
    if (i>>12 == 15) return true; // row 0
    int quartet = i>>8;         
    if ( (quartet - (quartet>>4<<4)) == 15 ) return(true); // row 1
    quartet = i>>4;
    if ( (quartet - (quartet>>4<<4)) == 15 ) return(true); // row 2
    quartet = i;
    if ( (quartet - (quartet>>4<<4)) == 15 ) return(true); // row 3    
    return(false);
}

__global__ void last_turn(float *outvec)
{
    int a;
    int ones = 0;
    // Идентификатор потока
    int i = threadIdx.x + blockIdx.x * blockDim.x;    
    // Убедимся, что число единиц и нулей в комбинации одинаково    
    for(a=0;a<16-1;a++) if ( ( (int)pow(2,a)&i)>0 ) ones++;
    // Проверим, победна ли комбинация
    if (ones==8 && win_combination(i)) outvec[i] = 1;
}
""")

In [4]:
combination_check_gpu = ker.get_function("last_turn")

combinations = np.zeros(256*256).astype(np.float32)
combinations_gpu_in = gpuarray.to_gpu(combinations)
combinations_gpu_out = gpuarray.empty_like(combinations_gpu_in)

%time combination_check_gpu( combinations_gpu_out,  block=(256,1,1), grid=(256,1,1))

CPU times: user 1.09 ms, sys: 0 ns, total: 1.09 ms
Wall time: 1.1 ms


In [5]:
out_combinations = combinations_gpu_out.get()

Количество победных комбинаций:

In [6]:
np.count_nonzero(out_combinations)

1482

Посмотрим, на победные комбинации.

In [7]:
def bin_interpretation(i):
    clear_bin = bin(i)[2:]
    lead_zeros = ''.join(['0' for z in range(16-len(clear_bin))])
    return lead_zeros+clear_bin

In [8]:
for i in np.where(out_combinations == 1)[0]:
    row = []
    i_bin = bin_interpretation(i)
    row.append(i_bin[:4])
    row.append(i_bin[4:8])
    row.append(i_bin[8:12])
    row.append(i_bin[12:])
    print('\n')
    for r in row:
        print(r)



0000
0000
0111
1111


0000
0000
1011
1111


0000
0000
1101
1111


0000
0000
1110
1111


0000
0000
1111
0111


0000
0000
1111
1011


0000
0000
1111
1101


0000
0000
1111
1110


0000
0001
0011
1111


0000
0001
0101
1111


0000
0001
0110
1111


0000
0001
1001
1111


0000
0001
1010
1111


0000
0001
1100
1111


0000
0001
1111
0011


0000
0001
1111
0101


0000
0001
1111
0110


0000
0001
1111
1001


0000
0001
1111
1010


0000
0001
1111
1100


0000
0010
0011
1111


0000
0010
0101
1111


0000
0010
0110
1111


0000
0010
1001
1111


0000
0010
1010
1111


0000
0010
1100
1111


0000
0010
1111
0011


0000
0010
1111
0101


0000
0010
1111
0110


0000
0010
1111
1001


0000
0010
1111
1010


0000
0010
1111
1100


0000
0011
0001
1111


0000
0011
0010
1111


0000
0011
0100
1111


0000
0011
1000
1111


0000
0011
1111
0001


0000
0011
1111
0010


0000
0011
1111
0100


0000
0011
1111
1000


0000
0100
0011
1111


0000
0100
0101
1111


0000
0100
0110
1111


0000
0100
1001
1111


0000
0100
1010
1111


0000
010



0010
1001
0001
1111


0010
1001
0010
1111


0010
1001
0100
1111


0010
1001
1000
1111


0010
1001
1111
0001


0010
1001
1111
0010


0010
1001
1111
0100


0010
1001
1111
1000


0010
1010
0001
1111


0010
1010
0010
1111


0010
1010
0100
1111


0010
1010
1000
1111


0010
1010
1111
0001


0010
1010
1111
0010


0010
1010
1111
0100


0010
1010
1111
1000


0010
1011
0000
1111


0010
1011
1111
0000


0010
1100
0001
1111


0010
1100
0010
1111


0010
1100
0100
1111


0010
1100
1000
1111


0010
1100
1111
0001


0010
1100
1111
0010


0010
1100
1111
0100


0010
1100
1111
1000


0010
1101
0000
1111


0010
1101
1111
0000


0010
1110
0000
1111


0010
1110
1111
0000


0010
1111
0000
0111


0010
1111
0000
1011


0010
1111
0000
1101


0010
1111
0000
1110


0010
1111
0001
0011


0010
1111
0001
0101


0010
1111
0001
0110


0010
1111
0001
1001


0010
1111
0001
1010


0010
1111
0001
1100


0010
1111
0010
0011


0010
1111
0010
0101


0010
1111
0010
0110


0010
1111
0010
1001


0010
1111
0010
1010


0010
111

0100
0010


0110
1111
0100
0100


0110
1111
0100
1000


0110
1111
0101
0000


0110
1111
0110
0000


0110
1111
1000
0001


0110
1111
1000
0010


0110
1111
1000
0100


0110
1111
1000
1000


0110
1111
1001
0000


0110
1111
1010
0000


0110
1111
1100
0000


0111
0000
0000
1111


0111
0000
1111
0000


0111
1000
0000
1111


0111
1000
1111
0000


0111
1111
0000
0001


0111
1111
0000
0010


0111
1111
0000
0100


0111
1111
0000
1000


0111
1111
0001
0000


0111
1111
0010
0000


0111
1111
0100
0000


0111
1111
1000
0000


1000
0000
0011
1111


1000
0000
0101
1111


1000
0000
0110
1111


1000
0000
1001
1111


1000
0000
1010
1111


1000
0000
1100
1111


1000
0000
1111
0011


1000
0000
1111
0101


1000
0000
1111
0110


1000
0000
1111
1001


1000
0000
1111
1010


1000
0000
1111
1100


1000
0001
0001
1111


1000
0001
0010
1111


1000
0001
0100
1111


1000
0001
1000
1111


1000
0001
1111
0001


1000
0001
1111
0010


1000
0001
1111
0100


1000
0001
1111
1000


1000
0010
0001
1111


1000
0010
0010
1111



1111
1010
0010
0100


1111
1010
0010
1000


1111
1010
0011
0000


1111
1010
0100
0001


1111
1010
0100
0010


1111
1010
0100
0100


1111
1010
0100
1000


1111
1010
0101
0000


1111
1010
0110
0000


1111
1010
1000
0001


1111
1010
1000
0010


1111
1010
1000
0100


1111
1010
1000
1000


1111
1010
1001
0000


1111
1010
1010
0000


1111
1010
1100
0000


1111
1011
0000
0001


1111
1011
0000
0010


1111
1011
0000
0100


1111
1011
0000
1000


1111
1011
0001
0000


1111
1011
0010
0000


1111
1011
0100
0000


1111
1011
1000
0000


1111
1100
0000
0011


1111
1100
0000
0101


1111
1100
0000
0110


1111
1100
0000
1001


1111
1100
0000
1010


1111
1100
0000
1100


1111
1100
0001
0001


1111
1100
0001
0010


1111
1100
0001
0100


1111
1100
0001
1000


1111
1100
0010
0001


1111
1100
0010
0010


1111
1100
0010
0100


1111
1100
0010
1000


1111
1100
0011
0000


1111
1100
0100
0001


1111
1100
0100
0010


1111
1100
0100
0100


1111
1100
0100
1000


1111
1100
0101
0000


1111
1100
0110
0000


1111
1100

In [9]:
i = np.where(out_combinations == 1)[0][0]
i

127

In [10]:
ones = 0
for a in range(16):
    if ( (2**a)&i)>0:
        ones += 1
ones

7